<a href="https://colab.research.google.com/github/CesarData/Coursera_Capstone_Project/blob/main/IBM_Capstone_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introducción
En este notebook se extrajeron los datos geográficos de los antiguos municipios que conforman la actual ciudad de Toronto (provincia de Ontario, Canadá): East York, Etobicoke, North York, Old Toronto, Scarborough y York. Los datos se extraen de la Wikipedia y con ellos creamos un dataframe que contiene los códigos postales, municipios, vecindarios y además se agregaron las coordenadas para su posterior visualización. Sin embargo, para este propósito, sólo elegimos Old Toronto que a su vez está dividido en East Toronto, Central Toronto, Downtown Toronto y West Toronto. Cada una de estas áreas es mostrada en el mapa con un color diferente. 

# Primera parte
##Extración de datos y creación de dataframe.

Importación de las librerías.

In [1]:
#Importación de librerías.

import requests # librería para manejar las solicitudes
import pandas as pd # librería para análisis de datos
import numpy as np # librería para manejar datos vectorizados
import io
import folium # librería para graficar 
from geopy.geocoders import Nominatim # módulo para convertir una dirección en valores de latitud y longitud 
from bs4 import BeautifulSoup

Extraemos la información de la Wikipedia para extraer la tabla donde se encuentran los datos. Debido a que el código postal esta en **Bold** podemos separarla fácilmente para almacenarlos en una lista *codes* tipo *bs4.element.ResultSet*. El resto de la información, es decir, municipio y vecindarios se encuentran etiquetados en un *span* y son almacenados en otra lista *boroughs_neighborhoods*. 

In [2]:
# Proceso de Web Scraping a través de Beautiful Soup

url_codes="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response=requests.get(url_codes)

soup = BeautifulSoup(response.content, "html.parser")
contentTable  = soup.find('table')
codes  = contentTable.findAll('b')
boroughs_neighborhoods = contentTable.findAll('span')

Se crean arreglos para los códigos postales, municipios y vencindarios además del dataframe que contiene todos estos datos. Los códigos postales de la lista *codes* se copian al arreglo correspondiente. Con la lista de *boroughs_neighborhoods* se hace un procesamiento para extraer el municipio y guardarlo en el arrego *boroughs* y en otro arreglo *neighborhoods* se guardan los vecindarios. Debido a que en algunas celdas se encuentran dos grupos de parentesis se reemplaza en primer parentesis que abre por un caracter asterisco (\*) para que este sea el parametro de la función *split* y así no perder la información contenida en el segundo grupo de parentesis que quedaba almacedada en la posición [i][2] del arreglo. Así sólo se divide la cadena en dos partes *boroughs* y *neighborhoods*.

In [3]:
# Creamos arrays y dataframes necesarios

df_toronto= pd.DataFrame() # dataframe que contiene los datos de la wikipedia, códdigo postal, municipios y vencindarios.
codes_array = np.array([]) # array de códigos postales.
boroughs_array = np.array([]) # array de municipios.
neighborhoods_array = np.array([]) # array de vecindarios.

# Rellenando arrays de códigos postales y vecindarios
for i in range(0, len(codes)):
    codes[i] = codes[i].get_text()
    boroughs_neighborhoods[i] = boroughs_neighborhoods[i].get_text()
    boroughs_neighborhoods[i] = boroughs_neighborhoods[i].replace("(", "*", 1) # Reemplazar carácter ( por * para no separar la cadena dos: boroughs y neighborhoods
    boroughs_neighborhoods[i] = boroughs_neighborhoods[i].split("*") 

    if boroughs_neighborhoods[i][0] != "Not assigned": # Solo procesando información válida
        
        boroughs_neighborhoods[i][1] = boroughs_neighborhoods[i][1].replace("(", ", ") # Eliminando paréntesis
        boroughs_neighborhoods[i][1] = boroughs_neighborhoods[i][1].replace(")", " ") # Eliminando paréntesis
        boroughs_neighborhoods[i][1] = boroughs_neighborhoods[i][1].replace(" /", ",") # Cambiar barra oblicua por comas
        
        codes_array = np.append(codes_array, codes[i])
        boroughs_array = np.append(boroughs_array, boroughs_neighborhoods[i][0])
        neighborhoods_array = np.append(neighborhoods_array, boroughs_neighborhoods[i][1])
      


# Añadiendo datos a columnas del dataframe
df_toronto['Postal Code'] = codes_array.tolist()
df_toronto['Borough'] = boroughs_array.tolist()
df_toronto['Neighborhood'] = neighborhoods_array.tolist()

df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
#Imprimimos el número de filas del dataframe
df_toronto.shape

(103, 3)

# Segunda parte
## Agregar coordenadas.

Se extraen las coordenadas de los códigos postales y se genera un dataframe para almacenarlas. Posteriormente se unen ambos dataframes en uno sólo que contiene toda la información.

In [6]:
# Extracción de las coordenadas. 

url_geo = 'http://cocl.us/Geospatial_data'
coor = requests.get(url_geo).text
df_coor = pd.read_csv(io.StringIO(coor))
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# Unión de los dataframes en uno sólo que contiene toda la información.

df_toronto_coor = pd.merge(df_toronto, df_coor, on = 'Postal Code')
df_toronto_coor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Tercera parte
## Generar mapa de Old Toronto.

Convertimos la dirección en valores de latitd y longitud.

In [14]:
# Extraer coordenadas de Toronto.

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#Imprimos las coordenadas de Toronto.
print(latitude, longitude)

43.6534817 -79.3839347


Usamos la función *unique* para ver un arreglo con los municipios contenidos en la columna *Borough* del dataframe.

In [15]:
# Extraer los nombres de los municipios.

df_toronto_coor.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East YorkEast Toronto', 'Central Toronto',
       'MississaugaCanada Post Gateway Processing Centre',
       'Downtown TorontoStn A PO Boxes25 The Esplanade',
       'EtobicokeNorthwest',
       'East TorontoBusiness reply mail Processing Centre969 Eastern'],
      dtype=object)

Creamos un dataframe que contiene sólo los datos de los municipios de Old Toronto, es decir, Downtown, East, West y Central Toronto.

In [16]:
#Crear dataframe con los datos de Old Toronto.

df_old_toronto = pd.DataFrame()
df_old_toronto = df_toronto_coor[df_toronto_coor['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_old_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Creamos un array de enteros que contiene valores del 0 al 3 que se usa para asignar un color diferente a cada municipio de Old Toronto. Este array se agrega al dataframe de Old Toronto.

In [18]:
# Añadir una columna al dataframe que contiene una clave de color para ser visualizado en el mapa.

# Arreglo que guarda la clave del color
chroma_array = np.array([], dtype=np.int8)

for i in range(0, len(df_old_toronto)):
  if 'downtown' in df_old_toronto['Borough'][i].lower():
    chroma_array = np.append(chroma_array, 0)
  elif 'east' in df_old_toronto['Borough'][i].lower():
    chroma_array = np.append(chroma_array, 1)
  elif 'west' in df_old_toronto['Borough'][i].lower():
    chroma_array = np.append(chroma_array, 2)
  elif 'central' in df_old_toronto['Borough'][i].lower():
    chroma_array = np.append(chroma_array, 3)

df_old_toronto['Chroma'] = chroma_array.tolist()
df_old_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Chroma
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0


Generamos un mapa donde se visualizan los municipios de Old Toronto en los siguientes colores:


*   Downtown Toronto - Darkblue
*   East Toronto - Green
*   West Toronto - Orange
*   Central Toronto - Purple

In [19]:
# Generar mapa con centro en Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13) 
colors = ['darkblue','green','orange','purple']

# Añadir los vecindarios
for lat, lng, borough, neighborhood, chroma in zip(df_old_toronto['Latitude'], df_old_toronto['Longitude'], df_old_toronto['Borough'], df_old_toronto['Neighborhood'], df_old_toronto['Chroma']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)  
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        icon=folium.Icon(color=colors[chroma]),
        color=colors[chroma],
        fill=True,
        fill_color = colors[chroma],
        fill_opacity=0.7,).add_to(map_toronto)  

# mostrar el mapa   
map_toronto

